In [1]:
import chromadb
from libraries.db import DatabaseProxy

In [2]:
chroma_client = chromadb.PersistentClient("./instance")
collection = chroma_client.create_collection(
    name="my_collection", metadata={"hnsw:space": "cosine"}, get_or_create=True
)

In [3]:
db = DatabaseProxy()

In [4]:
questions = db.get_questions()

In [5]:
documents = []
metadatas = []
ids = []
for question in questions:
    documents.append(question.text)
    metadatas.append({"id": question.id})
    ids.append(str(question.id))

In [6]:
if not collection.count():
    collection.add(ids, documents=documents, metadatas=metadatas)
collection.count()

300

In [7]:
questions[:5]

[Question(id=1, text='How many candidates do you have in your database?', answer_id=1, created_datetime=datetime.datetime(2023, 11, 8, 5, 16, 40)),
 Question(id=2, text='How often do you review and refresh your database of candidates?', answer_id=2, created_datetime=datetime.datetime(2023, 11, 8, 5, 16, 40)),
 Question(id=3, text='If you are a member of the American Staffing Association, what certifications do you hold?', answer_id=3, created_datetime=datetime.datetime(2023, 11, 8, 5, 16, 40)),
 Question(id=4, text='How do you determine which employees need coverage under the ACA?', answer_id=4, created_datetime=datetime.datetime(2023, 11, 8, 5, 16, 40)),
 Question(id=5, text='Can you commit to meet emergency requests and/or urgent ad-hoc requests from Segra?', answer_id=5, created_datetime=datetime.datetime(2023, 11, 8, 5, 16, 40))]

In [8]:
def print_results(results):
    for i in range(len(results["ids"][0])):
        print(round(results["distances"][0][i], 2), results["documents"][0][i])
        question = db.get_question_by_id(results["ids"][0][i])
        print(question.answer_id, db.get_answer_by_id(question.answer_id).text)
        print()

In [9]:
results = collection.query(query_texts=["How many employees work at your firm?"], n_results=5)
print_results(results)

0.35 Can you briefly explain the story of your company and the estimated employee count?
6 545 employees

0.35 What is the condensed version of your company's past and the total number of people working for you?
6 545 employees

0.42 Can you give a summary of your company's background and the size of your workforce?
6 545 employees

0.44 Provide a brief of company history and number of employees
6 545 employees

0.45 Could you provide a synopsis of your company's history and the approximate number of individuals on your team?
6 545 employees



In [13]:
results = collection.query(query_texts=["How often do you hire new employees?"], n_results=5)
print_results(results)

0.48 What action or steps do you implement to retain employees throughout the season?
47 Ex 1: We perform extensive screening for candidates that best match the job descriptions and are comfortable with the expectations at each site. We take care of all of our associates and over-communicate to ensure successful placement.                                                                                                                                                           
Ex 2: Employee Recognition
We are firm believers in recognizing and rewarding good behavior and have met with much success using some of these employee appreciation efforts 
ï¿½	Employee of the Month ï¿½ working together with supervisory staff in identifying deserving associates each month.
ï¿½	Raffle Employee Attendance Program.  Anyone with 100% attendance with 40 hours plus in a week, raffle items (gas cards, gift cards, CASH). 
ï¿½	Employee ï¿½Goalï¿½ Lunch or Raffle:  When an employee or ï¿½Teamï¿½ achieves a 

In [14]:
results = collection.query(
    query_texts=["How will WalMart be involved in the hiring and interview process?"], n_results=5
)
print_results(results)

0.62 Can you provide details on any additional screening methods utilized by your organization beyond employment verification and background checks?
46 Partners Personnel conducts Integrity testing, workplace safety training and testing, forklift safety videos and testing. Additionally, we conduct client specific orientations and customize our screening processes to the requirements of our customers.

0.63 In addition to employment verification and background checks, are there any other screening procedures your firm undertakes?
46 Partners Personnel conducts Integrity testing, workplace safety training and testing, forklift safety videos and testing. Additionally, we conduct client specific orientations and customize our screening processes to the requirements of our customers.

0.63 How does Partners Personnel conduct background screenings and what criteria do they use?
45 Partners Personnel standardly conducts background screenings from the Federal, National, State, County and City 

In [12]:
results = collection.query(query_texts=["Where do your employees like to go trick-or-treating?"], n_results=3)
print_results(results)

0.63 What actions do you implement to support and reward employees who consistently meet or exceed expectations?
47 Ex 1: We perform extensive screening for candidates that best match the job descriptions and are comfortable with the expectations at each site. We take care of all of our associates and over-communicate to ensure successful placement.                                                                                                                                                           
Ex 2: Employee Recognition
We are firm believers in recognizing and rewarding good behavior and have met with much success using some of these employee appreciation efforts 
ï¿½	Employee of the Month ï¿½ working together with supervisory staff in identifying deserving associates each month.
ï¿½	Raffle Employee Attendance Program.  Anyone with 100% attendance with 40 hours plus in a week, raffle items (gas cards, gift cards, CASH). 
ï¿½	Employee ï¿½Goalï¿½ Lunch or Raffle:  When an employe